# Progetto BigData
## Valerio Di Zio e Francesco Magnani

intro here

In [1]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
val bucketname = "unibo-bd2324-vdizio"

val path_book_sample = "s3a://"+bucketname+"/project/book_sample134.json"
val path_book_sample32row = "s3a://"+bucketname+"/project/book_sample32row.json"
val path_positive_words = "s3a://"+bucketname+"/project/positive-words.txt"
val path_negative_words = "s3a://"+bucketname+"/project/negative-words.txt"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1707556777915_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2324-vdizio
path_book_sample: String = s3a://unibo-bd2324-vdizio/project/book_sample134.json
path_book_sample32row: String = s3a://unibo-bd2324-vdizio/project/book_sample32row.json
path_positive_words: String = s3a://unibo-bd2324-vdizio/project/positive-words.txt
path_negative_words: String = s3a://unibo-bd2324-vdizio/project/negative-words.txt
res3: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1707556777915_0001/


In [101]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame


In [94]:
case class BookReview(
    id: String,
    overall:Double,
    reviewTime:String,
    reviewerID:String,
    asin:String,
    reviewText:String,
    summary:String
)

object BookReview{
    def extract(row:org.apache.spark.sql.Row) = {
        val overall = row.getAs[Double]("overall")
        val reviewTime = row.getAs[String]("reviewTime")
        val reviewerID = row.getAs[String]("reviewerID")
        val asin = row.getAs[String]("asin")
        val reviewText = row.getAs[String]("reviewText")
        val summary = row.getAs[String]("summary")
        val id = reviewerID + "-" + asin
    
        new BookReview(id, overall, reviewTime, reviewerID, asin, reviewText, summary)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class BookReview
defined object BookReview
Companions must be defined together; you may wish to use :paste mode for this.


In [150]:
val bookSampleRdd32row = spark.read.json(path_book_sample).map(BookReview.extract)
//val bookSampleRdd32row = spark.read.json(path_book_sample32row).map(BookReview.extract)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bookSampleRdd32row: org.apache.spark.sql.Dataset[BookReview] = [id: string, overall: double ... 5 more fields]


In [151]:
val positiveWords = sc.textFile(path_positive_words)
val negativeWords = sc.textFile(path_negative_words)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-vdizio/project/positive-words.txt MapPartitionsRDD[1092] at textFile at <console>:32
negativeWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-vdizio/project/negative-words.txt MapPartitionsRDD[1094] at textFile at <console>:32


In [152]:
def splitAndClean(id:String, text: String): Array[String] = 
    text.split("\\s+").map(el => id + "_" + el.toLowerCase.replaceAll("[^a-zA-Z\\s]", ""))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitAndClean: (id: String, text: String)Array[String]


In [153]:
val positiveDf = positiveWords.toDF("positiveWord")
val negativeDf = negativeWords.toDF("negativeWord")

def joinAndMapWords(review:DataFrame, wordsDataset: RDD[String]): RDD[(String, Int)] = {
    val df = wordsDataset.toDF("words")
    review.
        join(df, expr("SUBSTRING(word, instr(word, '_')+1, LEN(word)) like words")).
        map(row => (row.getAs[String](0), row.getAs[String](1))).
        map({case (el, _) => (el.substring(0, el.indexOf('_')), 1)}).
        rdd.
        reduceByKey(_ + _)
}
    

val reviewsRdd = bookSampleRdd32row.
    map(review => (review.id, review.reviewText)).
    filter(review => review._2 != null && review._2.length > 3).
    flatMap({case (id, text) => splitAndClean(id, text)}).
    filter(text => text.length > 3).
    toDF("word").
    cache()

val withPositiveRdd = joinAndMapWords(reviewsRdd, positiveWords)
    
val withNegativeRdd = joinAndMapWords(reviewsRdd, negativeWords)

val reducedResult = withPositiveRdd.
    fullOuterJoin(withNegativeRdd).
     map ({ case (key, (value1, value2)) => (key, (value1.getOrElse(0) - value2.getOrElse(0)))})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveDf: org.apache.spark.sql.DataFrame = [positiveWord: string]
negativeDf: org.apache.spark.sql.DataFrame = [negativeWord: string]
joinAndMapWords: (review: org.apache.spark.sql.DataFrame, wordsDataset: org.apache.spark.rdd.RDD[String])org.apache.spark.rdd.RDD[(String, Int)]
reviewsRdd: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [word: string]
withPositiveRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[1118] at reduceByKey at <console>:38
withNegativeRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[1132] at reduceByKey at <console>:38
reducedResult: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[1136] at map at <console>:35


In [ ]:
val resultDataframe = reducedResult.toDF("key", "value").cache()

val minValue = resultDataframe.agg(min("value")).head().getInt(0).toDouble
val maxValue = resultDataframe.agg(max("value")).head().getInt(0).toDouble

// a = min, b = max, c = 1 e d = 5
def mapIntToOverallVote(toMap: Double, fromMin: Double, fromMax: Double, toMin:Double=1, toMax:Double=5): Double =
     toMin + ((toMax-toMin)/(fromMax-fromMin))*(toMap-fromMin)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
val normalizedResult = 
resultDataframe.
    map({row => (row.getAs[String](0), mapIntToOverallVote(row.getAs[Int](1).toDouble, minValue, maxValue))}).
    rdd

In [ ]:
val relativeErrors = 
bookSampleRdd32row.
    map(review => (review.id, review.overall)).
    rdd.
    fullOuterJoin(normalizedResult).
    map ({ case (key, (real, estimated)) => 
            (key, Math.abs(real.getOrElse[Double](3) - estimated.getOrElse[Double](3)) / real.getOrElse[Double](3) )
        })

In [ ]:
val meanError = (relativeErrors.map(row => row._2).sum / relativeErrors.count)

println("Errore Medio: " + meanError*100 + "%")